In [51]:
'''
1. 지도는 5x525개의 위치로 이어지는 그리드 월드입니다.
2. 알파벳 R, G, B, Y는 4자리입니다.
3. 승객은 4개 위치 중 아무 곳에나 있을 수 있습니다.
4. 승객의 목적지는 왼쪽 3개 위치 중 하나일 수 있습니다.
5. 파이프 기호 |는 벽을 나타냅니다. 즉, |Y|x:위치 에 있으면 왼쪽으로 이동 x하여 위치에 도달할 수 없습니다 .Y
6. 콜론 기호 :는 패스를 나타냅니다. 즉, 위치에 |B:a|있는 경우 a왼쪽으로 이동하여 위치 B에 도달할 수 있습니다.
7. 택시는 통과할 수 :있지만 통과할 수 없습니다 .|
8. 20환경 은 승객이 목적지까지 내려갈 때 포인트를 보상 합니다.
9. -10승객이 없는 셀에서 픽업 작업을 수행하면 환경 에서 점수를 얻습니다.
10. -10승객이 택시에 탑승하지 않은 상태에서 하차 작업을 수행하면 환경에서 벌점 을 받습니다.
11. -1환경 은 다른 모든 행동에 불이익 을 줍니다.
12. 이 환경에는 500개의 상태가 있습니다.
'''

'\n1. 지도는 5x525개의 위치로 이어지는 그리드 월드입니다.\n2. 알파벳 R, G, B, Y는 4자리입니다.\n3. 승객은 4개 위치 중 아무 곳에나 있을 수 있습니다.\n4. 승객의 목적지는 왼쪽 3개 위치 중 하나일 수 있습니다.\n5. 파이프 기호 |는 벽을 나타냅니다. 즉, |Y|x:위치 에 있으면 왼쪽으로 이동 x하여 위치에 도달할 수 없습니다 .Y\n6. 콜론 기호 :는 패스를 나타냅니다. 즉, 위치에 |B:a|있는 경우 a왼쪽으로 이동하여 위치 B에 도달할 수 있습니다.\n7. 택시는 통과할 수 :있지만 통과할 수 없습니다 .|\n8. 20환경 은 승객이 목적지까지 내려갈 때 포인트를 보상 합니다.\n9. -10승객이 없는 셀에서 픽업 작업을 수행하면 환경 에서 점수를 얻습니다.\n10. -10승객이 택시에 탑승하지 않은 상태에서 하차 작업을 수행하면 환경에서 벌점 을 받습니다.\n11. -1환경 은 다른 모든 행동에 불이익 을 줍니다.\n12. 이 환경에는 500개의 상태가 있습니다.\n'

In [52]:
import numpy as np
import gym

In [53]:
def eps_greedy(Q, s, eps=0.1):
    
    # uniform → 균등 분포 정보를 생성
    if np.random.uniform(0,1) < eps:
        return np.random.randint(Q.shape[1])
    
    else:
        return greedy(Q, s)

In [54]:
# 축을 따라 최대값의 인덱스를 반환함
def greedy(Q, s):
    return np.argmax(Q[s])

In [55]:
def run_episodes(env, Q, num_episodes=100, to_print=False):
    
    tot_rew= []
    state = env.reset()
    
    for _ in range(num_episodes):
        done = False
        game_rew = 0
        
        while not done:
            next_state, rew, done, _ = env.step(greedy(Q, state))
            
            state = next_state
            game_rew += rew
            if done:
                state = env.reset()
                tot_rew.append(game_rew)
                
    if to_print:
        print('Mean score: %.3f of %i games!'%(np.mean(tot_rew), num_episodes))

In [56]:
def Q_learning(env, lr=0.01, num_episodes=10000, eps=0.3, gamma= 0.95, eps_decay= 0.00005):
    
    # action의 갯수
    nA = env.action_space.n
    # 차원의 수
    nS = env.observation_space.n
    
    Q = np.zeros((nS, nA))
    games_reward = []
    test_rewards = []
    
    for ep in range(num_episodes):
        state = env.reset()
        done = False
        tot_rew = 0
        
        # epsilon이 0.01에 닿을때까지 감쇠시킴
        if eps > 0.01:
            eps -= eps_decay
            
        while not done:
            
            action = eps_greedy(Q, state, eps)
            next_state, rew, done, _, info = env.step(action)
            # 상태- 동작 값 업데이트(다음 상태에 대한 최대 Q값 가져오기)
            Q[state][action] = Q[state][action] + lr * (rew + gamma * np.argmax(Q[next_state]) - Q[state][action])
            
            state = next_state
            tot_rew += rew
            if done:
                games_reward.apped(tot_rew)
        
        # 300번의 에피소드마다 점검함        
        if (ep % 300) == 0:
            test_rew = run_episodes(env, Q, 1000)
            print("Episode:{:5d}  Eps:{:2.4f}  Rew:{:2.4f}".format(ep, eps, test_rew))
            test_rewards.append(test_rew)
            
    return Q

In [57]:
# on-policy 알고리즘으로 Q-learning과 차별화되는 영역 중 하나
# 에이전트가 행동하고 가치 함수를 업데이트하는 데 동일한 정책을 사용함
# 정책 외 접근 방식에는 행동 및 업데이트에 대해 서로 다른 정책을 사용

def SARSA(env, lr=0.01, num_episodes =10000, eps=0.3, gamma=0.95, eps_decay=0.00005):
    
    nA = env.action_space.n
    nS = env.observation_space.n
    
    Q = np.zeros((nS, nA))
    games_reward = []
    test_rewards = []
    
    for ep in range(num_episodes):
        state = env.reset()
        done = False
        tot_rew = 0
        
        if eps > 0.01:
            eps -= eps_decay
            
        action =eps_greedy(Q, state, eps)
        
        # main body의 환경이 끝날때까지 loop
        while not done:
            next_state, rew, done, _, info = env.step(action)
            
            next_action = eps_greedy(Q, next_state, eps)
            
            Q[state][action] = Q[state][action] + lr * (rew+gamma * Q[next_state][next_action] - Q[state][action])
            
            state = next_state
            action = next_action
            tot_rew += rew
            
            if done:
                games_reward.append(tot_rew)
                
        if (ep % 300) == 0:
            test_rew = run_episodes(env, Q, 1000)
            print("Episode:{:5d}  Eps:{:2.4f}  Rew:{:2.4f}".format(ep, eps, test_rew))
            test_rewards.append(test_rew)
            
    return Q

In [58]:
if __name__ == '__main__':
    env = gym.make('Taxi-v3')
    
    Q_qlearning = Q_learning(env, lr=0.1, num_episodes =5000, eps=0.4, gamma = 0.95, eps_decay = 0.001)
    
    Q_sarsa = SARSA(env, lr=0.1, num_episodes=5000, eps=0.4, gamma=0.95, eps_decay=0.001)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices